## Definisi Masalah

Tenggelamnya kapal RMS Titanic adalah salah satu tragedi yang paling terkenal. Pada tanggal 15 April 1912, dalam pelayaran perdana ny, kapal Titanic tenggelam setelah bertabrakan dengan sebuah gunung es, mengakibatkan 1502 korban jiwa dari 2224 penumpan dan awak kapal. Tragedi ini sangat mengguncang komunitas internasional dan mendorong untuk membuat peraturan keselamatan yang lebih baik di dalam kapal dan pelayaran.

Salah satu alasan kapal Titanic mengakibatkan korban jiwa yang relatif banyak diakibatkan karena tidak cukupnya jumlah kapal penyelemat (skoci) bagi para penumpang dan awak kapal. Meskipun ada yang beruntung selamat pada saat proses penenggelaman, kelompok orang yang lebih banyak selamat dibandingkan yang lain antara lain para wanita, anak - anak, dan para kelas atas.

Dalam tugas ini, kita meminta anda untuk menyelesaikan analisis mengenai kriteria seperti apa orang yang selamat dari tragedi kapal Titanic. Khususnya, kita meminta anda untuk menerapkan <i>machine learning</i> untuk memprediksi penumpang mana saja yang selamat dari tragedi tersebut.

## Requirement

1. Tim pengajar telah menyiapkan data yang sudah dieksplorasi dan dibersihkan sehingga siap untuk dimasukkan ke dalam model
2. Tim pengajar telah menyiapkan 1 model yang sudah di-<i>training</i> dengan menggunakan algoritma <i>Random Forest</i>
3. Lakukan <i>Hyperparameter Search</i> dengan menggunakan <i>RandomizedSearchCV</i> dari <i>Scikit Learn</i>
4. Masukkan iterasi <i>Cross Validation</i>/CV ke dalam <i>Hyperparameter Search</i>, anda bisa memasukan angka 5 sebagai jumlah CV yang akan diterapkan

## Library Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split

c:\users\benedict\miniconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Import Dataset

In [2]:
df = pd.read_csv("./data/titanic.csv")
df.shape

(891, 8)

Berikut adalah deskripsi dari masing - masing kolom:
- survival = Selamat atau tidaknya penumpang yang bersangkutan (0 = Tidak, 1 = Selamat)
- pclass = Kelas tiket penumpang (1 = 1st, 2 = 2nd, 3 = 3rd)
- sex = Gender
- Age = Umur dalam tahun
- sibsp = Jumlah saudara kandung/pasangan yang bersama dengan penumpang bersangkutan di atas kapal
- parch = Jumlah orang tua/anak - anak yang bersama dengan penumpang bersangkutan di atsa kapal
- fare = Jumlah biaya perjalanan penumpang
- cabin = Nomor kabin
- embarked = Nama pelabuhan tempat penumpang berangkat

In [3]:
df.head()

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived
0,22,S,7.2500,0,3,male,1,0.0
1,38,C,71.2833,0,1,female,1,1.0
2,26,S,7.9250,0,3,female,0,1.0
3,35,S,53.1000,0,1,female,1,1.0
4,35,S,8.0500,0,3,male,0,0.0


## Preprocessing

In [4]:
numeric_data = df.loc[:, ["Age", "Fare", "Parch", "SibSp"]].copy()
categorical_data = df.loc[:, ["Embarked", "Pclass", "Sex",]].copy()

In [5]:
# Standard Scaling numerical data
sc = StandardScaler()
numeric_data = sc.fit_transform(numeric_data)
numeric_data.shape

(891, 4)

In [6]:
# Label Binarizer categorical data
embarked_dummy = pd.get_dummies(categorical_data["Embarked"], prefix="embarked")
categorical_data = categorical_data.drop(["Embarked"], axis=1)
categorical_data = pd.concat([categorical_data, embarked_dummy], axis=1)

pclass_dummy = pd.get_dummies(categorical_data["Pclass"], prefix="pclass")
categorical_data = categorical_data.drop(["Pclass"], axis=1)
categorical_data = pd.concat([categorical_data, pclass_dummy], axis=1)

sex_dummy = pd.get_dummies(categorical_data["Sex"], prefix="sex")
categorical_data = categorical_data.drop(["Sex"], axis=1)
categorical_data = pd.concat([categorical_data, sex_dummy], axis=1)

categorical_data.head()

,embarked_C,embarked_Q,embarked_S,pclass_1,pclass_2,pclass_3,sex_female,sex_male
0,0,0,1,0,0,1,0,1
1,1,0,0,1,0,0,1,0
2,0,0,1,0,0,1,1,0
3,0,0,1,1,0,0,1,0
4,0,0,1,0,0,1,0,1


In [7]:
# Combine the preprocessed numerical data & categorical data
df = df.loc[:, ["Age", "Fare", "Parch", "SibSp", "Survived"]].copy()
df.loc[:, "Age":"SibSp"] = numeric_data
df = pd.concat([df, categorical_data], axis=1)
df.head()

,Age,Fare,Parch,SibSp,Survived,embarked_C,embarked_Q,embarked_S,pclass_1,pclass_2,pclass_3,sex_female,sex_male
0,-0.510769,-0.502445,-0.473674,0.432793,0.0,0,0,1,0,0,1,0,1
1,0.579769,0.786845,-0.473674,0.432793,1.0,1,0,0,1,0,0,1,0
2,-0.238134,-0.488854,-0.473674,-0.474545,1.0,0,0,1,0,0,1,1,0
3,0.375293,0.420730,-0.473674,0.432793,1.0,0,0,1,1,0,0,1,0
4,0.375293,-0.486337,-0.473674,-0.474545,0.0,0,0,1,0,0,1,0,1


In [8]:
# Train test split
X = df.drop(["Survived"], axis=1)
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

## Random Forest Classifier

In [9]:
filename = "./model/rf.sav"
rf_model = pickle.load(open(filename, 'rb'))
rf_model

c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=123, verbose=1, warm_start=False)

In [10]:
pred = rf_model.predict(X_test)

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished


In [11]:
from sklearn.metrics import f1_score

f1_score(y_test, pred)

0.7719298245614034

## Hyperparameter Search & CV (100 point)

Lakukan <i>Hyperparameter Search</i> & CV untuk meningkatkan performa (F1 score) model yang sudah ada. Bandingkan F1 score yang dihasilkan sebelum melakukan <i>Hyperparameter Search</i> & CV

### Get current parameters of model

In [12]:
rf_model.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 123,
 'verbose': 1,
 'warm_start': False}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

rf_tuned_params = {'min_samples_split' : [2, 3, 5, 10],
                  'n_estimators' : [1000, 2000, 3000],
                   'max_depth' : [1, 2, 3, 4], 
                   'min_samples_leaf': [1, 3, 5]}

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

n_iter_search = 50
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=rf_tuned_params,
                                   n_iter=n_iter_search,cv=5)


random_search.fit(X,y)

In [ ]:
print("Best Params : ",random_search.best_params_)
print()
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

### Apply Optimized Params

In [44]:
Optimized_model=RandomForestClassifier(bootstrap= False, criterion= 'entropy', max_depth= None, max_features= 9, min_samples_leaf= 8, min_samples_split= 5) 

Optimized_model.fit(X_train,y_train)
y_opt = Optimized_model.predict(X_test)

## Compare Results :

In [45]:
print("F1 Score Before Hyperparameter Search CV : ",f1_score(y_test, pred))
print("F1 Score After Hyperparameter Search CV : ",f1_score(y_test, y_opt))

F1 Score Before Hyperparameter Search CV :  0.7719298245614034
F1 Score After Hyperparameter Search CV :  0.7796610169491526
